# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Sharma  ->  Y. M. Sharma  |  ['M. Sharma']
D. Ostertag  ->  D. Ostertag  |  ['D. Ostertag']
M. Flock  ->  M. Flock  |  ['M. Flock']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
Arxiv has 49 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2501.18034


extracting tarball to tmp_2501.18034... done.
Retrieving document from  https://arxiv.org/e-print/2501.18424


extracting tarball to tmp_2501.18424...

 done.


D. Ostertag  ->  D. Ostertag  |  ['D. Ostertag']
M. Flock  ->  M. Flock  |  ['M. Flock']


Found 39 bibliographic references in tmp_2501.18424/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2501.18477
extracting tarball to tmp_2501.18477...

 done.


E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']


Found 167 bibliographic references in tmp_2501.18477/K2-18b.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.18424-b31b1b.svg)](https://arxiv.org/abs/2501.18424) | **Strong clumping in global streaming instability simulations with a dusty fluid**  |
|| <mark>D. Ostertag</mark>, <mark>M. Flock</mark> |
|*Appeared on*| *2025-01-31*|
|*Comments*| *7 pages, 6 figures*|
|**Abstract**|            Context: How planets form in protoplanetary disks and what drives the formation of their seeds is still a major unknown. It is an accepted theory that multiple processes can trap dusty material in radially narrow rings or vortex-like structures, preventing the dust from drifting inwards. However, the relevant process for clumping this dusty material until it collapses under gravity still needs to be identified. One promising candidate is the streaming instability arising from the aerodynamic interaction between dust and gas once they reach similar densities. Aims: We investigate with a global disk model based on recent observational constraints if streaming instability can form dust clumps, which might gravitationally collapse. Further, our goal is to verify the observability of the produced structures using ALMA or ngVLA. Methods. For the first time, we present global 2D (R, z) hydrodynamic simulations using FARGO3D in which the dust is treated as a pressureless fluid. The disk model assumes stratification, realistic boundary conditions, and meaningful resolution to resolve the fast-growing modes. We choose two values for the total dust-to-gas mass ratio Z = 0.01 and Z = 0.02, compare the maximum clump density to the local Hill density, and compute the optical depth of the dust disk. Results: With a dust-to-gas mass ratio of Z = 0.01, we confirm previous streaming instability simulations, not showing the ability to form strong concentrations of dust clumps. With Z = 0.02, dense clumps form within 20 orbits, however reaching only 30% of the Hill density even following disk parameters from the massive protoplanetary disks GM Aur, HD163296, IM Lup, MWC 480, and TW Hya, which all share astonishingly similar surface density profiles.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.18477-b31b1b.svg)](https://arxiv.org/abs/2501.18477) | **A Comprehensive Reanalysis of K2-18 b's JWST NIRISS+NIRSpec Transmission Spectrum**  |
|| S. P. Schmidt, et al. -- incl., <mark>E.-M. Ahrer</mark> |
|*Appeared on*| *2025-01-31*|
|*Comments*| *42 pages, 20 figures. Submitted to AAS Journals*|
|**Abstract**|            Sub-Neptunes are the most common type of planet in our galaxy. Interior structure models suggest that the coldest sub-Neptunes could host liquid water oceans underneath their hydrogen envelopes - sometimes called 'hycean' planets. JWST transmission spectra of the $\sim$ 250 K sub-Neptune K2-18 b were recently used to report detections of CH$_4$ and CO$_2$, alongside weaker evidence of (CH$_3$)$_2$S (dimethyl sulfide, or DMS). Atmospheric CO$_2$ was interpreted as evidence for a liquid water ocean, while DMS was highlighted as a potential biomarker. However, these notable claims were derived using a single data reduction and retrieval modeling framework, which did not allow for standard robustness tests. Here we present a comprehensive reanalysis of K2-18 b's JWST NIRISS SOSS and NIRSpec G395H transmission spectra, including the first analysis of the second-order NIRISS SOSS data. We incorporate multiple well-tested data reduction pipelines and retrieval codes, spanning 60 different data treatments and over 250 atmospheric retrievals. We confirm the detection of CH$_4$ ($\approx$ 4$\sigma$), with a volume mixing ratio of log CH$_4$ = $-1.15^{+0.40}_{-0.52}$, but we find no statistically significant or reliable evidence for CO$_2$ or DMS. Finally, we quantify the observed atmospheric composition using photochemical-climate and interior models, demonstrating that our revised composition of K2-18 b can be explained by an oxygen-poor mini-Neptune without requiring a liquid water surface or life.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.18034-b31b1b.svg)](https://arxiv.org/abs/2501.18034) | **An energetic absorption outflow in QSO J1402+2330: Analysis of DESI observations**  |
|| M. Dehghanian, et al. -- incl., <mark>M. Sharma</mark> |
|*Appeared on*| *2025-01-31*|
|*Comments*| *8 pages, 7 figures, accepted for publication in A&A*|
|**Abstract**|            Context. Quasar outflows play a significant role in the active galactic nucleus (AGN) feedback, impacting the interstellar medium and potentially influencing galaxy evolution. Characterizing these outflows is essential for understanding AGN-driven processes. Aims. We aim to analyze the physical properties of the mini-broad absorption line outflow in quasar J1402+2330 using data from the Dark Energy Spectroscopic Instrument (DESI) survey. We seek to measure the outflows location, energetics, and potential impact on AGN feedback processes. Methods. In the spectrum of J1402+2330, we identify multiple ionic absorption lines, including ground and excited states. We measure the ionic column densities and then use photoionization models to determine the total hydrogen column density and ionization parameter of the outflow. We utilized the population ratio of the excited state to the ground state of N iii and S iv to determine the electron number density. Results. The derived electron number density, combined with the ionization parameter, indicates an outflow distance of approximately 2200 pc from the central source. Having a mass outflow rate of more than one thousand solar masses per year and a kinetic energy output exceeding 5 percent of the Eddington luminosity, this outflow can significantly contribute to AGN feedback. Conclusions. Our findings suggest the absorption outflow in J1402+2330 plays a potentially significant role in AGN feedback processes. This study highlights the value of DESI data in exploring AGN feedback mechanisms.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2501.18424/./plots/DensityProfileComparisonHillDensity.png', 'tmp_2501.18424/./plots/dustDensity2D_EvolDTG_4096x1024_Z002_FARGO.png', 'tmp_2501.18424/./plots/evolutionMaxDustDensity_4096x1024_Z001Z002.png']
copying  tmp_2501.18424/./plots/DensityProfileComparisonHillDensity.png to _build/html/
copying  tmp_2501.18424/./plots/dustDensity2D_EvolDTG_4096x1024_Z002_FARGO.png to _build/html/
copying  tmp_2501.18424/./plots/evolutionMaxDustDensity_4096x1024_Z001Z002.png to _build/html/
exported in  _build/html/2501.18424.md
    + _build/html/tmp_2501.18424/./plots/DensityProfileComparisonHillDensity.png
    + _build/html/tmp_2501.18424/./plots/dustDensity2D_EvolDTG_4096x1024_Z002_FARGO.png
    + _build/html/tmp_2501.18424/./plots/evolutionMaxDustDensity_4096x1024_Z001Z002.png
found figures ['tmp_2501.18477/./plot/K2-18b_Detection_Significance_Grids.png', 'tmp_2501.18477/./plot/K2-18b_Retrieval_Average_Posterior.png', 'tmp_2501.18477/./plot/Nep-100Xsolar.png', 'tmp_2501.18

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Strong clumping in global streaming instability simulations with a dusty fluid

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.18424-b31b1b.svg)](https://arxiv.org/abs/2501.18424)<mark>Appeared on: 2025-01-31</mark> -  _7 pages, 6 figures_

</div>
<div id="authors">

<mark>D. Ostertag</mark>, <mark>M. Flock</mark>

</div>
<div id="abstract">

**Abstract:** How planets form in protoplanetary disks and what drives the formation of their seeds is still a major unknown. It is an accepted theory that multiple processes can trap dusty material in radially narrow rings or vortex-like structures, preventing the dust from drifting inwards. However, the relevant process for clumping this dusty material until it collapses under gravity still needs to be identified. One promising candidate is the streaming instability arising from the aerodynamic interaction between dust and gas once they reach similar densities. We investigate with a global disk model based on recent observational constraints if streaming instability can form dust clumps, which might gravitationally collapse. Further, our goal is to verify the observability of the produced structures using ALMA or ngVLA. For the first time, we present global 2D ( $R,z$ ) hydrodynamic simulations using FARGO3D in which the dust is treated as a pressureless fluid. The disk model assumes stratification, realistic boundary conditions, and meaningful resolution to resolve the fast-growing modes. We choose two values for the total dust-to-gas mass ratio $Z=0.01$ and $Z=0.02$ , compare the maximum clump density to the local Hill density, and compute the optical depth of the dust disk. With a dust-to-gas mass ratio of $Z=0.01$ , we confirm previous streaming instability simulations, not showing the ability to form strong concentrations of dust clumps.With $Z=0.02$ , dense clumps form within 20 orbits, however reaching only 30 $\%$ of the Hill density even following disk parameters from the massive protoplanetary disks GM Aur, HD163296, IM Lup, MWC 480, and TW Hya, which all share an astonishing similar surface density profiles. Our results show that clumping by the streaming instability to trigger self-gravity is less efficient than previously thought, especially when applying more realistic density profiles. By extrapolating our results, we estimate the gravitational collapse of concentrated pebbles earliest at 480 orbits, while for more frequent, less massive, or more compact disks, this time can reach 1000 orbits.Our results predict that substructures caused by streaming instability can vary between optical thin and optical thick at ALMA Band 1 wavelength for less massive disks.However, the average clump separation is 0.03 au at 10 au distance to the star, far too small to be observable with ALMA and even ngVLA.For the currently observed disks and best-fit surface density profiles, we predict efficient planetesimal formation outside 10 au, where the ratio of Hill- and gas midplane density is sufficiently small. Our results suggest that even for massive Class II disks, the critical Hill density can be reached in dust concentrations during 480 - 1000 orbits, corresponding to tens or hundreds of thousands of years, depending on the radial position.

</div>

<div id="div_fig1">

<img src="tmp_2501.18424/./plots/DensityProfileComparisonHillDensity.png" alt="Fig3" width="100%"/>

**Figure 3. -** Comparison between five midplane gas density profiles (grey) and their critical Hill densities (blue). For reference, the red dot indicates our work's central location and gas density. A similar plot, though with only one gas density profile, was already shown in [ and Klahr (2020)](). (*fig:densityProfileComparison*)

</div>
<div id="div_fig2">

<img src="tmp_2501.18424/./plots/dustDensity2D_EvolDTG_4096x1024_Z002_FARGO.png" alt="Fig1" width="100%"/>

**Figure 1. -** Evolution of the disk with $\Sigma_{\mathrm{d}}/\Sigma_{\mathrm{g}}=0.02$. Top: 2D dust density after 160 orbits, bottom: Evolution of the surface density ratio over 160 orbits. The white dashed line shows the theoretical radial drift with a constant drift velocity of $v_{\mathrm{r, clump}}$ = \qty{30}{cm.s^{-1}}. (*fig:EvolutionDTGRatio_Z002*)

</div>
<div id="div_fig3">

<img src="tmp_2501.18424/./plots/evolutionMaxDustDensity_4096x1024_Z001Z002.png" alt="Fig2" width="100%"/>

**Figure 2. -** Evolution of the maximum dust density between 9.4 and 10.4 au over 160 orbits for the considered simulations. The black line indicates the density ratio that clumps need to reach to collapse under their own gravity. The domain is chosen not to consider clumps that are an artifact from the outer boundary. (*fig:evolutionMaxDustDensity*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.18424"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\eureka}{\texttt{Eureka!}\xspace}$</div>



<div id="title">

# A Comprehensive Reanalysis of K2-18 b's JWST NIRISS+NIRSpec Transmission Spectrum

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.18477-b31b1b.svg)](https://arxiv.org/abs/2501.18477)<mark>Appeared on: 2025-01-31</mark> -  _42 pages, 20 figures. Submitted to AAS Journals_

</div>
<div id="authors">

S. P. Schmidt, et al. -- incl., <mark>E.-M. Ahrer</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ Sub-Neptunes are the most common type of planet in our galaxy. Interior structure models suggest that the coldest sub-Neptunes could host liquid water oceans underneath their hydrogen envelopes---sometimes called "hycean" planets. JWST transmission spectra of the $\sim$ 250 K sub-Neptune K2-18 b were recently used to report detections of $CH_4$ and $CO_2$ , alongside weaker evidence of ($CH_3$ ) $_2$ S (dimethyl sulfide, or DMS). Atmospheric $CO_2$ was interpreted as evidence for a liquid water ocean, while DMS was highlighted as a potential biomarker. However, these notable claims were derived using a single data reduction and retrieval modeling framework, which did not allow for standard robustness tests. Here we present a comprehensive reanalysis of K2-18 b's JWST NIRISS SOSS and NIRSpec G395H transmission spectra, including the first analysis of the second-order NIRISS SOSS data. We incorporate multiple well-tested data reduction pipelines and retrieval codes, spanning 60 different data treatments and over 250 atmospheric retrievals. We confirm the detection of $CH_4$ ( $\approx 4\sigma$ ), with a volume mixing ratio of $\log \mathrm{CH_4} = -1.15^{+0.40}_{-0.52}$ , but we find no statistically significant or reliable evidence for $CO_2$ or DMS. Finally, we quantify the observed atmospheric composition using photochemical-climate and interior models, demonstrating that our revised composition of K2-18 b can be explained by an oxygen-poor mini-Neptune without requiring a liquid water surface or life.

</div>

<div id="div_fig1">

<img src="tmp_2501.18477/./plot/K2-18b_Detection_Significance_Grids.png" alt="Fig1" width="100%"/>

**Figure 1. -** Detection significances of $CH_4$, $CO_2$, and DMS from \texttt{POSEIDON} retrievals of K2-18 b's JWST NIRISS SOSS + NIRSpec G395H transmission spectrum. Each grid cell represents a Bayesian model comparison between two retrievals, one with and one without a molecule, for a specific NIRISS SOSS and NIRSpec G395H data variant pair. The data variants are ordered with increasing spectral resolution to the right (for NIRSpec) and down (for NIRISS). The cells are colored and annotated according to the detection significance equivalent to the calculated Bayes factor \citep[e.g.,][]{Trotta2008,Benneke2013}. Bayes factors $<3$(equivalent to $< 2.1\sigma$), corresponding to no evidence on the Jeffreys' scale, are donated by `---'. Our retrievals demonstrate strong evidence for $CH_4$, but find no reliable statistical evidence for $CO_2$ or DMS in K2-18 b's atmosphere.
     (*fig:retrieval_detection_significances*)

</div>
<div id="div_fig2">

<img src="tmp_2501.18477/./plot/K2-18b_Retrieval_Average_Posterior.png" alt="Fig13" width="100%"/>

**Figure 13. -** Atmospheric, planetary, and stellar properties retrieved from K2-18 b's JWST NIRISS SOSS + NIRSpec G395H transmission spectrum by \texttt{POSEIDON}. Top block: retrieved atmospheric $\log_{\rm{10}}$ volume mixing ratios. Middle row: retrieved 10 mbar temperature, planetary mass, and cloud-top pressure. Bottom row: retrieved starspot (stellar heterogeneity) coverage fraction, starspot temperature, and stellar photosphere temperature. Parameters with clear upper and lower limits are annotated with the median and $\pm 1\sigma$ credible interval (vertical and dashed lines). Non-detected molecules are annotated with upper limits representing the 95\% credible interval (solid lines and left-pointing arrows). The non-detected cloud pressure is annotated with the lower limit representing the 5\% credible interval (solid line with right-pointing arrow). The \texttt{POSEIDON} posteriors shown here are the combined retrieval results from 60 NIRISS SOSS and NIRSpec G395H data variants (spanning different reduction codes and final spectral resolution); thus, they represent the marginalized posterior over all data-level analysis choices considered in this study. (*fig:retrieval_average_posterior*)

</div>
<div id="div_fig3">

<img src="tmp_2501.18477/./plot/Nep-100Xsolar.png" alt="Fig3.1" width="50%"/><img src="tmp_2501.18477/./plot/Nep-100Xsolar-00002XN.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** Self-consistent mini-Neptune models of K2-18 b's atmosphere. Upper panel: 100$\times$ solar metallicity case. Lower panel: the same metallicity, but with 5000 $\times$ nitrogen depletion. The temperature (black lines) and composition profiles (colored lines) from \texttt{VULCAN}-\texttt{HELIOS}(solid lines) and \texttt{Photochem}-\texttt{PICASO}(dashed lines) with the 1 $\sigma$ $CH_4$ constraint (orange error bar) and 95\% upper limits for other molecules (arrows) from the ensemble  retrieval posterior (Figure \ref{fig:retrieval_average_posterior}). (*fig:Nep_clouds*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.18477"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

324  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
